In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
#! git clone https://github.com/Tikquuss/meta_XLM /content/meta_XLM

In [ ]:
#! cp -R /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs /content

In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [2]:
#%cd tools
#! git clone https://github.com/moses-smt/mosesdecoder
#! git clone https://github.com/glample/fastBPE && cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

In [3]:
#%cd -b HOME

In [4]:
%env OUTPATH=/home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [9]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [10]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed/Ghomala-Limbum.Ghomala.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6333


In [11]:
%env epoch_size=6333

%env lgs=Ghomala-Limbum

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6333
env: lgs=Ghomala-Limbum
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [11]:
%env mlm_steps=Ghomala,Limbum,Ghomala-Limbum
%env batch_size=8
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster3
! python train.py --exp_name mlm_tlm_GhomalaLimbum --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Ghomala,Limbum,Ghomala-Limbum
env: batch_size=8
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster3
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/20/20 08:46:41 - 0:00:00 - ============ Initialized logger ============
INFO - 05/20/20 08:46:41 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 8
                                     beam_size: 1
                     

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_GhomalaLimbum  /content/drive/"My Drive"/African_Translator/models/africa/cluster3

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [5]:
%env epoch_size=6360
%env lgs=Ghomala-Limbum

env: epoch_size=6360
env: lgs=Ghomala-Limbum


In [6]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [7]:
%env stopping_criterion=valid_Ghomala-Limbum_mt_bleu,10
%env validation_metrics=valid_Ghomala-Limbum_mt_bleu
#%env reload_model=dumped/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth
%env reload_model=/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth

%env ae_steps=Ghomala,Limbum
%env bt_steps=Ghomala-Limbum-Ghomala,Limbum-Ghomala-Limbum

env: stopping_criterion=valid_Ghomala-Limbum_mt_bleu,10
env: validation_metrics=valid_Ghomala-Limbum_mt_bleu
env: reload_model=/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth
env: ae_steps=Ghomala,Limbum
env: bt_steps=Ghomala-Limbum-Ghomala,Limbum-Ghomala-Limbum


In [ ]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100


# unsupervised MT
#! python train.py --exp_name UnSupMT_GhomalaLimbum --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [ ]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env batch_size=32
%env max_epoch=100

%env dump_path=/home/jupyter/models/africa/cluster3

# supervised MT
%env mt_steps=Ghomala-Limbum,Limbum-Ghomala          
! python train.py --exp_name SupMT_GhomalaLimbum --exp_id $exp_id  --dump_path $dump_path --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: batch_size=32
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster3
env: mt_steps=Ghomala-Limbum,Limbum-Ghomala
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/21/20 08:47:05 - 0:00:00 - ============ Initialized logger ============
INFO - 05/21/20 08:47:05 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Ghomala', 'Limbum']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                

INFO - 05/21/20 08:47:05 - 0:00:00 - 256647 words (5057 unique) in 6332 sentences. 0 unknown words (0 unique) covering 0.00% of the data.
INFO - 05/21/20 08:47:05 - 0:00:00 - Removed 0 empty sentences.
INFO - 05/21/20 08:47:05 - 0:00:00 - Removed 40 too long sentences.

INFO - 05/21/20 08:47:05 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed/valid.Limbum.pth ...
INFO - 05/21/20 08:47:05 - 0:00:00 - 32483 words (5057 unique) in 791 sentences. 113 unknown words (83 unique) covering 0.35% of the data.
INFO - 05/21/20 08:47:05 - 0:00:00 - Removed 0 too long sentences.

INFO - 05/21/20 08:47:05 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed/test.Limbum.pth ...
INFO - 05/21/20 08:47:05 - 0:00:00 - 32593 words (5057 unique) in 791 sentences. 87 unknown words (75 unique) covering 0.27% of the data.
INFO - 05/21/20 08:47:05 - 0:00:00 - Removed 4 too long sentences.


INFO - 05/21/20 08:47:05 - 0:0

INFO - 05/21/20 08:47:10 - 0:00:05 - Number of parameters (encoder): 81289153
INFO - 05/21/20 08:47:10 - 0:00:05 - Number of parameters (decoder): 106491841
INFO - 05/21/20 08:47:10 - 0:00:05 - Found 0 memories.
INFO - 05/21/20 08:47:10 - 0:00:05 - Found 12 FFN.
INFO - 05/21/20 08:47:10 - 0:00:05 - Found 264 parameters in model.
INFO - 05/21/20 08:47:10 - 0:00:05 - Optimizers: model
WARNING - 05/21/20 08:47:10 - 0:00:05 - Reloading checkpoint from /home/jupyter/models/africa/cluster3/SupMT_GhomalaLimbum/maml/checkpoint.pth ...
WARNING - 05/21/20 08:47:12 - 0:00:07 - Not reloading checkpoint optimizer model.
WARNING - 05/21/20 08:47:12 - 0:00:07 - Reloading 'num_updates' and 'lr' for optimizer model.
WARNING - 05/21/20 08:47:12 - 0:00:07 - Checkpoint reloaded. Resuming at epoch 13 / iteration 442 ...
INFO - 05/21/20 08:47:12 - 0:00:08 - ============ Starting epoch 13 ... ============
INFO - 05/21/20 08:47:12 - 0:00:08 - Creating new training data iterator (ae,Limbum) ...
INFO - 05/21/20

INFO - 05/21/20 08:58:10 - 0:11:05 -     500 -   17.54 sent/s -  7871.14 words/s - AE-Ghomala:  1.4063 || AE-Limbum:  1.2706 || MT-Ghomala-Limbum:  1.7743 || MT-Limbum-Ghomala:  2.0201 || BT-Ghomala-Limbum-Ghomala:  2.2226 || BT-Limbum-Ghomala-Limbum:  1.9652 -  - model LR: 7.5025e-05
INFO - 05/21/20 08:59:08 - 0:12:03 -     505 -   22.86 sent/s - 10542.05 words/s - AE-Ghomala:  1.3692 || AE-Limbum:  1.3549 || MT-Ghomala-Limbum:  1.9538 || MT-Limbum-Ghomala:  1.9963 || BT-Ghomala-Limbum-Ghomala:  2.1515 || BT-Limbum-Ghomala-Limbum:  2.0207 -  - model LR: 7.5774e-05
INFO - 05/21/20 09:00:14 - 0:13:10 -     510 -   23.00 sent/s -  9845.53 words/s - AE-Ghomala:  1.3822 || AE-Limbum:  1.2455 || MT-Ghomala-Limbum:  1.7811 || MT-Limbum-Ghomala:  2.0750 || BT-Ghomala-Limbum-Ghomala:  2.1836 || BT-Limbum-Ghomala-Limbum:  1.9165 -  - model LR: 7.6524e-05
INFO - 05/21/20 09:00:14 - 0:13:10 - ============ End of epoch 14 ============
INFO - 05/21/20 09:01:31 - 0:14:27 - BLEU /home/jupyter/models/

INFO - 05/21/20 09:17:16 - 0:30:12 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 09:17:16 - 0:30:12 - ============ Starting epoch 16 ... ============
INFO - 05/21/20 09:17:29 - 0:30:24 -     545 -    3.93 sent/s -  3032.13 words/s - AE-Ghomala:  1.3161 || AE-Limbum:  1.1664 || MT-Ghomala-Limbum:  1.8591 || MT-Limbum-Ghomala:  2.0851 || BT-Ghomala-Limbum-Ghomala:  2.1438 || BT-Limbum-Ghomala-Limbum:  1.7791 -  - model LR: 8.1768e-05
INFO - 05/21/20 09:18:31 - 0:31:26 -     550 -   18.43 sent/s - 16861.73 words/s - AE-Ghomala:  1.2680 || AE-Limbum:  1.0900 || MT-Ghomala-Limbum:  1.8432 || MT-Limbum-Ghomala:  2.0686 || BT-Ghomala-Limbum-Ghomala:  2.1184 || BT-Limbum-Ghomala-Limbum:  1.9277 -  - model LR: 8.2518e-05
INFO - 05/21/20 09:18:59 - 0:31:55 - Creating new training data iterator (ae,Ghomala) ...
INFO - 05/21/20 09:19:08 - 0:32:04 - Creating new training data iterator (bt,Ghomala) ...
INFO - 05/21/20 09:19:40 - 0:32:36 -     555 -   18.16 sent/s - 15730.28 words

...

INFO - 05/21/20 18:52:30 - 10:05:25 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaLimbum/maml/hypotheses/hyp72.Ghomala-Limbum.test.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaLimbum/maml/hypotheses/ref.Ghomala-Limbum.test.txt : 18.380000
INFO - 05/21/20 18:52:30 - 10:05:25 - epoch -> 72.000000
INFO - 05/21/20 18:52:30 - 10:05:25 - valid_Limbum-Ghomala_mt_ppl -> 136.981023
INFO - 05/21/20 18:52:30 - 10:05:25 - valid_Limbum-Ghomala_mt_acc -> 40.739518
INFO - 05/21/20 18:52:30 - 10:05:25 - valid_Limbum-Ghomala_mt_bleu -> 11.130000
INFO - 05/21/20 18:52:30 - 10:05:25 - valid_Ghomala-Limbum_mt_ppl -> 49.804244
INFO - 05/21/20 18:52:30 - 10:05:25 - valid_Ghomala-Limbum_mt_acc -> 47.637795
INFO - 05/21/20 18:52:30 - 10:05:25 - valid_Ghomala-Limbum_mt_bleu -> 16.250000
INFO - 05/21/20 18:52:30 - 10:05:25 - test_Limbum-Ghomala_mt_ppl -> 118.057314
INFO - 05/21/20 18:52:30 - 10:05:25 - test_Limbum-Ghomala_mt_acc -> 41.509291
INFO - 05/21/20 18:52:30 - 10:05:25 - test_Limbum-

INFO - 05/21/20 19:07:18 - 10:20:13 -    2520 -    3.91 sent/s - 55896.67 words/s - AE-Ghomala:  0.0928 || AE-Limbum:  0.0680 || MT-Ghomala-Limbum:  0.0590 || MT-Limbum-Ghomala:  0.0376 || BT-Ghomala-Limbum-Ghomala:  0.0558 || BT-Limbum-Ghomala-Limbum:  0.0590 -  - model LR: 5.1434e-05
INFO - 05/21/20 19:08:16 - 10:21:11 -    2525 -   21.92 sent/s - 348093.67 words/s - AE-Ghomala:  0.0830 || AE-Limbum:  0.0866 || MT-Ghomala-Limbum:  0.0511 || MT-Limbum-Ghomala:  0.0459 || BT-Ghomala-Limbum-Ghomala:  0.0595 || BT-Limbum-Ghomala-Limbum:  0.0547 -  - model LR: 5.1383e-05
INFO - 05/21/20 19:09:32 - 10:22:27 -    2530 -   18.85 sent/s - 267408.39 words/s - AE-Ghomala:  0.1091 || AE-Limbum:  0.0764 || MT-Ghomala-Limbum:  0.0507 || MT-Limbum-Ghomala:  0.0494 || BT-Ghomala-Limbum-Ghomala:  0.0538 || BT-Limbum-Ghomala-Limbum:  0.0557 -  - model LR: 5.1333e-05
INFO - 05/21/20 19:10:05 - 10:23:01 - Creating new training data iterator (ae,Ghomala) ...
INFO - 05/21/20 19:10:15 - 10:23:11 - Creating

INFO - 05/21/20 19:32:26 - 10:45:22 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaLimbum/maml/hypotheses/hyp75.Ghomala-Limbum.test.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaLimbum/maml/hypotheses/ref.Ghomala-Limbum.test.txt : 18.020000
INFO - 05/21/20 19:32:26 - 10:45:22 - epoch -> 75.000000
INFO - 05/21/20 19:32:26 - 10:45:22 - valid_Limbum-Ghomala_mt_ppl -> 138.784479
INFO - 05/21/20 19:32:26 - 10:45:22 - valid_Limbum-Ghomala_mt_acc -> 40.457063
INFO - 05/21/20 19:32:26 - 10:45:22 - valid_Limbum-Ghomala_mt_bleu -> 11.130000
INFO - 05/21/20 19:32:26 - 10:45:22 - valid_Ghomala-Limbum_mt_ppl -> 51.731010
INFO - 05/21/20 19:32:26 - 10:45:22 - valid_Ghomala-Limbum_mt_acc -> 47.634790
INFO - 05/21/20 19:32:26 - 10:45:22 - valid_Ghomala-Limbum_mt_bleu -> 15.570000
INFO - 05/21/20 19:32:26 - 10:45:22 - test_Limbum-Ghomala_mt_ppl -> 117.585030
INFO - 05/21/20 19:32:26 - 10:45:22 - test_Limbum-Ghomala_mt_acc -> 41.630641
INFO - 05/21/20 19:32:26 - 10:45:22 - test_Limbum-

INFO - 05/21/20 19:47:43 - 11:00:38 -    2625 -   15.16 sent/s - 272903.32 words/s - AE-Ghomala:  0.0665 || AE-Limbum:  0.0595 || MT-Ghomala-Limbum:  0.0396 || MT-Limbum-Ghomala:  0.0381 || BT-Ghomala-Limbum-Ghomala:  0.0479 || BT-Limbum-Ghomala-Limbum:  0.0477 -  - model LR: 5.0395e-05
INFO - 05/21/20 19:48:56 - 11:01:51 -    2630 -   16.57 sent/s - 291256.33 words/s - AE-Ghomala:  0.0747 || AE-Limbum:  0.0783 || MT-Ghomala-Limbum:  0.0405 || MT-Limbum-Ghomala:  0.0378 || BT-Ghomala-Limbum-Ghomala:  0.0461 || BT-Limbum-Ghomala-Limbum:  0.0495 -  - model LR: 5.0347e-05
INFO - 05/21/20 19:49:57 - 11:02:53 -    2635 -   20.90 sent/s - 344324.91 words/s - AE-Ghomala:  0.0824 || AE-Limbum:  0.0789 || MT-Ghomala-Limbum:  0.0506 || MT-Limbum-Ghomala:  0.0374 || BT-Ghomala-Limbum-Ghomala:  0.0493 || BT-Limbum-Ghomala-Limbum:  0.0541 -  - model LR: 5.0300e-05
INFO - 05/21/20 19:51:13 - 11:04:08 -    2640 -   16.65 sent/s - 282083.61 words/s - AE-Ghomala:  0.0815 || AE-Limbum:  0.0628 || MT-Gho

INFO - 05/21/20 20:12:51 - 11:25:47 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 20:12:51 - 11:25:47 - ============ Starting epoch 79 ... ============
INFO - 05/21/20 20:13:55 - 11:26:50 -    2690 -    3.75 sent/s - 57910.78 words/s - AE-Ghomala:  0.0696 || AE-Limbum:  0.0919 || MT-Ghomala-Limbum:  0.0430 || MT-Limbum-Ghomala:  0.0422 || BT-Ghomala-Limbum-Ghomala:  0.0467 || BT-Limbum-Ghomala-Limbum:  0.0590 -  - model LR: 4.9783e-05
INFO - 05/21/20 20:14:57 - 11:27:52 -    2695 -   21.53 sent/s - 352037.85 words/s - AE-Ghomala:  0.0745 || AE-Limbum:  0.0768 || MT-Ghomala-Limbum:  0.0422 || MT-Limbum-Ghomala:  0.0371 || BT-Ghomala-Limbum-Ghomala:  0.0538 || BT-Limbum-Ghomala-Limbum:  0.0556 -  - model LR: 4.9736e-05
INFO - 05/21/20 20:16:14 - 11:29:10 -    2700 -   16.83 sent/s - 282656.57 words/s - AE-Ghomala:  0.0682 || AE-Limbum:  0.0734 || MT-Ghomala-Limbum:  0.0409 || MT-Limbum-Ghomala:  0.0407 || BT-Ghomala-Limbum-Ghomala:  0.0444 || BT-Limbum-Ghomala-Limbum:

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/SupMT_GhomalaLimbum  /content/drive/"My Drive"/African_Translator/models/africa/cluster3